# <center>Brief Générateur de paroles de chansons</center>

## Importation des librairies

In [3]:
import numpy as np 
import pandas as pd

In [4]:
with open('personal_access_token.txt') as f:
    ACCESS_TOKEN = f.readline()

## Importation des datasets

In [5]:
!git clone -b jordan https://{ACCESS_TOKEN}@github.com/jnamor/nlp_generator>.git

fatal: destination path 'nlp_generator' already exists and is not an empty directory.


In [6]:
df = pd.read_csv('nlp_generator/notebook/new-dataset.csv', sep=",")
df.head()

,Artist,Genres,Songs,Popularity,Link,SName,SLink,Lyric,language
0,4 Non Blondes,Rock,15.0,10.1,/4-non-blondes/,What's Up,/4-non-blondes/whats-up.html,Twenty-five years and my life is still\nTrying...,en
1,4 Non Blondes,Rock,15.0,10.1,/4-non-blondes/,Spaceman,/4-non-blondes/spaceman.html,Starry night bring me down\nTill I realize the...,en
2,4 Non Blondes,Rock,15.0,10.1,/4-non-blondes/,Pleasantly Blue,/4-non-blondes/pleasantly-blue.html,Every time you wake in the mornin'\nAnd you st...,en
3,4 Non Blondes,Rock,15.0,10.1,/4-non-blondes/,I'm The One,/4-non-blondes/im-the-one.html,Ah-hah!\nWoo!\nAh-ha-ha-ha-ha-ha!\nWe came her...,en
4,4 Non Blondes,Rock,15.0,10.1,/4-non-blondes/,Dear Mr. President,/4-non-blondes/dear-mr-president.html,I'm looking outside of my windows\nThe view th...,en


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4672 entries, 0 to 4671
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Artist      4672 non-null   object 
 1   Genres      4672 non-null   object 
 2   Songs       4672 non-null   float64
 3   Popularity  4672 non-null   float64
 4   Link        4672 non-null   object 
 5   SName       4672 non-null   object 
 6   SLink       4672 non-null   object 
 7   Lyric       4672 non-null   object 
 8   language    4672 non-null   object 
dtypes: float64(2), object(7)
memory usage: 328.6+ KB


In [8]:
df.shape

(4672, 9)

## Pre-processing
Maintenant que les données sont filtrées, nous allons effectuer un traitement qui permettra à notre modèle de prédire un suite de mots.

In [9]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
df = df.sample(1000)

In [11]:
token = Tokenizer()
token.fit_on_texts(df['Lyric'])

In [12]:
encoded_text = token.texts_to_sequences(df['Lyric'])

In [13]:
vocab_size = len(token.word_counts) + 1

In [14]:
datalist = []
for d in encoded_text:
    if len(d)>1:
        for i in range(2, len(d)):
            datalist.append(d[:i])

In [15]:
max_length = 20
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')
X = sequences[:, :-1]
y = sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#it will take sometime to complete traning
model.fit(X, y, batch_size=32, epochs=50)

In [ ]:
poetry_length = 10
def generate_poetry(seed_text, n_lines):
    for i in range(n_lines):
        text = []
        for _ in range(poetry_length):
            encoded = token.texts_to_sequences([seed_text])
            encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

            y_pred = np.argmax(model.predict(encoded), axis=-1)

            predicted_word = ""
            for word, index in token.word_index.items():
                if index == y_pred:
                    predicted_word = word
                    break
                    
            seed_text = seed_text + ' ' + predicted_word
            text.append(predicted_word)
                
        seed_text = text[-1]
        text = ' '.join(text)
        print(text)

In [ ]:
seed_text = 'i love you'
generate_poetry(seed_text, 5)